In [2]:
from termcolor import colored 
import os
import subprocess
try:
    import time 
    import json 
    import requests
    print(colored("[ x ]    ","green"),"Libraries Imported succesfully.")
except Exception as e:
    print(colored("[ x ]   ","red"   ),"Library Importing failed.")

[ x ]     Libraries Imported succesfully.


#### Checking if Ollama is ready or not

In [5]:
OLLAMA_URL  =   "https://localhost:11434/v1/chat/comletions"
MODEL       =   "deepseek-coder:6.7b"

def check_ollama_ready():
    try:
        response = requests.get("http://localhost:11434/api/tags")
        if response.status_code != 200:
            print(colored("[ x ]", "red"), "Ollama server not responding")
            return False
        if response.status_code == 200:
            print(colored("[ x ]    ","green"), "Ollama server running.")
        models = response.json().get('models', [])
        model_exists = any(m['name'].startswith(MODEL.split(':')[0]) for m in models)
        
        if not model_exists:
            print(colored("[ x ]", "red"), f"Model {MODEL} not found. Pulling it now...")
            import subprocess
            subprocess.run(["ollama", "pull", MODEL])
            print(colored("[ ✓ ]", "green"), "Model pulled successfully")
        
        return True

    except requests.exceptions.ConnectionError:
        print(colored("[ x ]", "red"), "Cannot connect to Ollama. Run 'ollama serve'")
        return False



In [6]:
check_ollama_ready()

[ x ]     Ollama server running.


True

In [ ]:
def stream_generate(conversation):
    payload = {
        "model":MODEL,
        "message": conversation,
        "stream":True,
        "options":{
            "temperature":0.7,
            "num_predict":500
        }
    }
    full_response = ""
    
    try:
        with requests.post(OLLAMA_URL, json=payload, stream=True, timeout=60) as r:
            r.raise_for_status()
            
            print(f"\nAssistant:", end=" ", flush=True)
            for line in r.iter_line():
                if not line:
                    continue
                
                line = line.decode("utf-8")
                
                if line.startswith("data: "):
                    line = line[6:]
                if line.strip() == "[DONE]":
                    break 
                
                try:
                    data = json.loads(line)
                    delta = data["choices"][0]["delta"]
                    if "content" in delta:
                        token = delta["content"]
                        full_response += token 
                        print(token, end="", flush=True)
                        
                except json.JSONDecodeError:
                    continue 
            print("\n")
        return full_response
    except Exception as e:
        print(colored("[ x ]    ","red"), f"ERROR: {e}")
        return "" 
    
    
def main():
    if not check_ollama_ready():
        return

    time.sleep(1)

    print(colored("Interactive Chat Started (type 'exit 0' to quit)\n", "cyan"))

    conversation = []

    while True:
        user_input = input("You: ")

        if user_input.strip().lower() == "exit 0":
            print("Exiting...")
            break

        conversation.append({"role": "user", "content": user_input})

        response = stream_generate(conversation)

        conversation.append({"role": "assistant", "content": response})


if __name__ == "__main__":
    main()    
            

[ x ]     Ollama server running.
Interactive Chat Started (type 'exit 0' to quit)

[ x ]     ERROR: HTTPSConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /v1/chat/comletions (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1032)')))
